In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt # 득점모델 변수 중요도
import xgboost


import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
%matplotlib inline

In [36]:
pd.set_option('display.max_columns', None)

In [37]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [38]:
##train데이터 불러오기
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [39]:
train_df.drop([ 'X_48', 'X_47', 'X_23', 'X_11', 'X_10', 'X_04'], inplace= True, axis=1)
test_df.drop([ 'X_48', 'X_47', 'X_23', 'X_11', 'X_10', 'X_04'], inplace= True, axis=1)

In [40]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y_01 Feature..
test_x = test_df.filter(regex='X')
test_y = test_df.filter(regex='Y')

In [41]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6, init='k-means++', max_iter=300, random_state=42) # K-means
kmeans.fit(train_x)

KMeans(n_clusters=6, random_state=42)

In [42]:
kmm_cluster_labels_tr = kmeans.predict(train_x)
kmm_cluster_labels_te = kmeans.predict(test_x)

In [43]:
train_x["km_label"] = kmm_cluster_labels_tr
test_x["km_label"] = kmm_cluster_labels_te

C:\Users\User\AppData\Local\Temp\ipykernel_13624\641694845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x["km_label"] = kmm_cluster_labels_tr


## PCA

In [149]:
pca_list = [['X_01','X_02','X_05','X_06'],['X_14','X_15','X_16','X_17','X_18'],['X_19','X_20','X_21','X_22'],['X_30','X_31','X_32','X_33'],['X_24','X_25','X_26','X_27','X_28','X_29'],['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]

In [52]:
from sklearn.preprocessing import MaxAbsScaler
train_col = train_x.columns
train_x = MaxAbsScaler().fit_transform(train_x)
train_x = pd.DataFrame(data = train_x, columns=train_col)

In [153]:
pca = PCA(n_components=1)
cnt = 1
pca_result = []
for i in pca_list:
    pca_x = train_x[i]
    printcipalComponents = pca.fit_transform(pca_x)
    pca_result.append(printcipalComponents)
    cnt += 1

In [154]:
cnt =1
for i in pca_result:
    train_x[f"{cnt}_X_comp"] = i
    cnt += 1

In [155]:
for i in pca_list:
    train_x.drop(i, inplace = True, axis=1)

In [46]:
train_x

,X_01,X_02,X_03,X_05,X_06,X_07,X_08,X_09,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,km_label
0,0.831691,0.99999,0.756645,0.987708,0.859709,0.179727,0.026128,0.385433,0.966592,0.642857,0.988881,0.990370,0.984570,0.993387,0.990420,0.829333,0.863760,0.831522,0.825858,0.872340,0.880851,0.872340,0.876596,0.876596,0.995763,0.706161,0.234397,0.595918,0.222791,0.993119,0.983957,0.984721,0.993119,-0.509153,-0.957285,-0.961061,0.980574,0.979011,0.993928,0.989212,0.690476,0.995916,0.084722,0.842726,0.695887,0.852394,0.881284,0.717528,0.803506,0.805192,0.6
1,0.819665,1.00000,0.730851,0.988212,0.836320,0.175333,0.025647,0.366453,0.975501,0.642857,0.988139,0.987407,0.988979,0.992652,0.988946,0.792000,0.847411,0.790761,0.844327,0.893617,0.893617,0.893617,0.893617,0.923404,0.966102,0.706161,0.231623,0.591837,0.212548,0.987768,0.983193,0.984721,0.988532,-0.498294,-0.942657,-0.944695,0.978723,0.980877,0.988323,0.991088,0.309524,0.995916,0.090984,0.822390,0.699078,0.864425,0.811189,0.730598,0.842302,0.804220,0.6
2,0.855730,0.99999,0.718515,0.999932,0.836320,0.175821,0.044303,0.426987,0.971047,0.535714,0.990363,0.989630,0.983835,0.992652,0.989683,0.810667,0.828338,0.817935,0.823219,0.880851,0.876596,0.868085,0.880851,0.893617,0.957627,0.706161,0.234397,0.595918,0.215109,0.991590,0.983193,0.983193,0.993884,-0.501396,-0.946167,-0.945824,0.977336,0.980877,0.988790,0.990619,0.333333,0.999319,0.095567,0.816662,0.673761,0.846493,0.846020,0.725565,0.825100,0.787224,0.6
3,0.843704,0.99999,0.757766,0.988474,0.883087,0.176492,0.048257,0.400571,0.964365,0.750000,0.985915,0.987407,0.983835,0.992652,0.987472,0.813333,0.820163,0.820652,0.812665,0.876596,0.872340,0.863830,0.876596,0.880851,0.898305,0.696682,0.233010,0.600000,0.215109,0.987003,0.990833,0.992361,0.987768,-0.497983,-0.937975,-0.940745,0.979648,0.978545,0.990191,0.989212,0.523810,1.000000,0.130993,0.824863,0.684989,0.805588,0.867815,0.746764,0.867585,0.868607,1.0
4,0.819665,0.99999,0.712908,0.988571,0.812942,0.181130,0.043302,0.378767,0.968820,0.571429,0.989622,0.988148,0.985305,0.991918,0.988946,0.810667,0.836512,0.815217,0.823219,0.889362,0.876596,0.889362,0.902128,0.991489,0.902542,0.706161,0.233010,0.600000,0.233035,0.990826,0.981665,0.986249,0.990826,-0.504189,-0.949678,-0.950903,0.978723,0.977612,0.989257,0.989681,0.523810,1.000000,0.096462,0.877687,0.702369,0.777529,0.884215,0.717770,0.818843,0.793887,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,0.783601,0.99999,0.698329,0.999903,0.766175,0.184304,0.032600,0.467537,0.971047,0.535714,0.991105,0.991111,0.992652,0.993387,0.991894,0.853333,0.825613,0.831522,0.825858,0.893617,0.897872,0.885106,0.897872,0.893617,0.932203,0.649289,0.221914,0.555102,0.213828,0.985474,0.993125,0.988541,0.986239,-0.499224,-0.944997,-0.947517,0.979186,0.983209,0.989724,0.993902,0.261905,1.000000,0.529232,0.799202,0.672485,0.769619,0.821966,0.695681,0.786220,0.830963,0.4
39603,0.783601,1.00000,0.703936,0.988959,0.766175,0.178262,0.042828,0.424587,0.979955,0.464286,0.990363,0.989630,0.989713,0.991183,0.990420,0.840000,0.833787,0.828804,0.807388,0.902128,0.902128,0.889362,0.902128,0.897872,0.932203,0.663507,0.233010,0.559184,0.226633,0.984709,0.983957,0.984721,0.983180,-0.487124,-0.921006,-0.925508,0.978723,0.983675,0.990659,0.993902,0.285714,0.992512,0.530390,0.784862,0.617741,0.822584,0.803544,0.702572,0.840939,0.790916,0.4
39604,0.807640,0.99999,0.725244,0.999845,0.789553,0.182839,0.042979,0.310703,0.975501,0.500000,0.991846,0.991852,0.989713,0.993387,0.991894,0.861333,0.841962,0.834239,0.823219,0.906383,0.889362,0.885106,0.880851,0.902128,0.915254,0.658768,0.223301,0.559184,0.202305,0.992355,0.991597,0.991597,0.979358,-0.500155,-0.947

In [50]:
xgb_model = xgboost.XGBRegressor(n_estimators=1000, max_depth=8, learning_rate=0.01, n_jobs=-1, gamma= 0.2)
XGB_Ro = MultiOutputRegressor(xgb_model).fit(train_x, train_y)

In [51]:
train_predict = XGB_Ro.predict(train_x)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, train_y))))

RMSE':0.9475551520556567


In [10]:
train_predict = XGB_Ro.predict(train_x)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, train_y))))

RMSE':0.9492028144734755


In [11]:
pd.DataFrame(train_predict,columns = train_y.columns).describe()

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
count,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000
mean,1.353747,1.057187,1.013932,13.620844,31.289715,16.529612,3.154878,-26.293447,-26.307249,-22.398649,24.324232,-26.236353,-26.232475,-26.244484
std,0.116527,0.121496,0.108348,0.937612,0.732126,1.342139,0.127677,0.230039,0.221863,0.389638,0.238047,0.224976,0.224004,0.225433
min,0.404497,0.351872,0.299945,7.646999,23.501703,-18.324881,1.327926,-27.919693,-27.916819,-29.310888,21.024137,-27.957317,-27.902531,-27.928865
25%,1.285587,0.985413,0.949705,13.104576,30.941003,16.489721,3.086912,-26.438828,-26.442783,-22.517544,24.207579,-26.374919,-26.369854,-26.382521
50%,1.356715,1.058575,1.015577,13.586365,31.353821,16.595230,3.158880,-26.288960,-26.306171,-22.318678,24.325808,-26.234173,-26.229675,-26.241232
75%,1.423892,1.127850,1.077948,14.106930,31.721309,16.711881,3.224092,-26.161982,-26.184808,-22.185302,24.449680,-26.110471,-26.106593,-26.119935
max,2.893292,2.794436,2.366600,74.396576,34.850319,18.107565,4.169287,-24.676090,-24.691492,-20.388151,25.677616,-24.491632,-24.673759,-24.739145


In [170]:
loss_fold = []
kfold = KFold(n_splits=3, shuffle=True)
for train, test in kfold.split(train_x):
    XGB = MultiOutputRegressor(xgb_model).fit(train_x.loc[train], train_y.loc[train])
    train_predict = XGB.predict(train_x.loc[test])
    scores = math.sqrt(mean_squared_error(train_predict, train_y.loc[test]))
    loss_fold.append(scores)

In [171]:
loss_fold

[1.1968468203154572, 1.2109570822549804, 1.1965353319503536]